# Preamble

In [7]:
# json
import json

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
from torch import Tensor

import matplotlib.pyplot as plt
import librosa

import librosa.display
import IPython.display as ipd

import numpy as np
import pickle
import random
import torch
import librosa
from torch_specinv import griffin_lim
from torch_specinv.metrics import spectral_convergence as SC

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn import metrics
import youtube_dl
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

import cvxpy as cp
import math
from torchaudio import functional as AF

In [8]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  True


In [9]:
FIXED_SAMPLE_RATE = 48000

# Load data

In [32]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from classification.models.DeepRecursiveCNN import DeepRecursiveCNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
pickle_path = "/nfs/students/summer-term-2020/project-4/data/dataset1/dataset_resampled/"
training = pickle.load(open(pickle_path + "training.p","rb"))
validation = pickle.load(open(pickle_path + "validation.p","rb"))

In [11]:
max_length = 0

for sample in training:
    max_length = max(max_length, sample['data'][0].shape[0])

for sample in validation:
    max_length = max(max_length, sample['data'][0].shape[0])
    
print(max_length)

481489


In [12]:
def prepareData(data):
    zero_padded_data = torch.zeros(max_length)
    zero_padded_data[:data.shape[0]] = torch.from_numpy(data)
    return zero_padded_data

In [13]:
training_dataset = [(prepareData(sample['data'][0]), 1)
                        if sample['binary_class']=='positive' else (prepareData(sample['data'][0]), 0) 
                        for sample in training]
validation_dataset = [(prepareData(sample['data'][0]), 1)
                        if sample['binary_class']=='positive' else (prepareData(sample['data'][0]), 0) 
                        for sample in validation]

In [71]:
path_tum_sound = '/nfs/students/summer-term-2020/project-4/yan/tum.wav'
tum_sound,sr = librosa.load(path_tum_sound, sr=FIXED_SAMPLE_RATE)
padding = int((max_length - len(tum_sound))/2)
zero_padded_data = torch.zeros(max_length)
zero_padded_data[padding:padding+tum_sound.shape[0]] = torch.from_numpy(tum_sound)
tum_sound = zero_padded_data

# Model aspects

## Definition

In [14]:
def validate(model, epoch=0):
    valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    
    model.eval()

    y_true = []
    y_pred = []
    running_loss = 0.0
    
    with torch.no_grad():
        for i,data in enumerate(valid_loader,0):
            samples, labels = [x.cuda() for x in data]
            outputs = model(samples)

            y_pred.append(torch.max(outputs.data, 1)[1].item())
            y_true.append(labels.item())
            
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
    print('[%d] valid-loss: %.3f' % (epoch + 1, running_loss / (i+1)))
            
    return reportScore(y_true, y_pred), running_loss / (i+1)

In [15]:
def reportScore(y_true, y_pred):
    print("\tAccuracy:\t" + str(metrics.accuracy_score(y_true,y_pred)))
    print("\tPrecision:\t" + str(metrics.precision_score(y_true,y_pred)))
    print("\tRecall:   \t" + str(metrics.recall_score(y_true,y_pred)))
    print("\tF1-score:\t" + str(metrics.f1_score(y_true,y_pred)))

    #tn, fp, fn, tp = metrics.confusion_matrix(y_true,y_pred).ravel()
    return metrics.accuracy_score(y_true,y_pred)

## Loading model

In [33]:
model_state_dict_path = "/nfs/students/summer-term-2020/project-4/yan/models/best_model_state_dict.pt"
model = DeepRecursiveCNN()
model.load_state_dict(torch.load(model_state_dict_path))
model.eval() # important! we don't optimize the model anymore

DeepRecursiveCNN(
  (bn0): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv1d(1, 128, kernel_size=(80,), stride=(4,))
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.2, inplace=False)
  (conv2): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(256, 512, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(512, 1024, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

# Attacks

In [14]:
windowsize = 2048
window = torch.hann_window(windowsize)

def audio2spec(y):
    return torch.stft(y, windowsize, window=window).pow(2).sum(2).sqrt()

def drawSpec(mag):
    log_S = librosa.power_to_db(mag.numpy(), ref=np.max(mag.numpy()))
    plt.figure(figsize=(12,2))
    ax = librosa.display.specshow(log_S, sr=FIXED_SAMPLE_RATE, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')

In [93]:
from attacks.fgsm import FGSM

## New attack

In [15]:
## speedup with padding
def time_stretch(sample, speedup_rate):
    if speedup_rate == 1:
        return sample
    
    n_fft = 2048 # windowsize
    hop_length = int(np.floor(n_fft / 4))
    
    # speedup
    stft = torch.stft(sample, n_fft, hop_length=hop_length, window=window).unsqueeze(0)
    phase_advance = torch.linspace(0, math.pi * hop_length, stft.shape[1])[..., None]
    vocoded = AF.phase_vocoder(stft, rate=speedup_rate, phase_advance=phase_advance)
    istft = AF.istft(vocoded, n_fft, hop_length=hop_length, window=window).squeeze()
    
    max_length = sample.shape[0]

    if speedup_rate > 1: 
        # faster means output is smaller -> padding
        pad_l = int((max_length - istft.shape[0])/2)
        pad_r = max_length - (pad_l + istft.shape[0])
        return F.pad(istft, (pad_l, pad_r))
    else:
        # slower means longer -> chopping of
        low = int((istft.shape[0] - max_length)/2)
        return istft[low:low+max_length]

In [84]:
def speed_attack(model, x, y, epsilon, num_iter=1):
    a=0.9
    b=1.1
    rate_search_range = torch.arange(a, b, float(1)/num_iter)
    losses = []
    stretched_inputs = []
    
    with torch.no_grad():
        for rate in rate_search_range:
            stretched = time_stretch(x.squeeze().cpu(), rate).cuda().unsqueeze(0)
            stretched_inputs.append(stretched)
            losses.append(F.nll_loss(model(stretched), y))
            
    best_rate = torch.stack(losses).argmax().item()
    return stretched_inputs[best_rate]

In [66]:
torch.utils.data.DataLoader(audioset)

In [82]:
valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
acc, ex = testAttack(model, valid_loader, speed_attack, -1, num_iter=20, early_stopping=-1)

 43%|████▎     | 732/1687 [38:56<50:48,  3.19s/it]  


KeyboardInterrupt: 

In [77]:
ex

[(7, 1, 0, array([0., 0., 0., ..., 0., 0., 0.], dtype=float32))]

In [79]:
eps = -1
adversarial_samples = {}

for sample in ex:
    if sample[1] == 1: # ie. previously correctly classified as EM
        print("found good adversarial sample: ")
        example_id = sample[0]
        print("\tSample id: " + str(example_id))
        adversarial_samples.update({example_id : sample})

found good adversarial sample: 
	Sample id: 7


In [ ]:
sample = adversarial_samples[7] #random.sample(list(adversarial_samples.values()),1)[0]
print(sample[0])
original = validation_dataset[sample[0]][0]
adversarial = sample[-1]

ipd.display(ipd.Audio(original,    rate=FIXED_SAMPLE_RATE, normalize=False))
ipd.display(ipd.Audio(adversarial, rate=FIXED_SAMPLE_RATE, normalize=False))

In [ ]:
raise Exception("hi")

## Interpolation

In [ ]:
'''
    Interpolation attack: 
        - take gradient wrt. the interpolation parameters a,b 
        
    Parameters:
        - tum_sound: sound to be inserted
        - epsilon: gradient step size
        - num_iterations: PGD iterations
        - clamping parameters for a, b (4 in total): define the max/min of interpolation volume
'''
def insertion_attack(model, x, y, epsilon, num_iter=1):
    global tum_sound
    
    x = x.clone().detach()
    a = torch.tensor(1.0).cuda() # original sound volume
    b = torch.tensor(1.0).cuda() # inserted sound volume
    tum_sound = tum_sound.cuda()
    
    for i in range(num_iter):
        a.requires_grad_()
        b.requires_grad_()
        
        loss = F.nll_loss(model(a * x + b * tum_sound), y)
        model.zero_grad()
        loss.backward()
        
        a = (a + epsilon * a.grad.data).clamp(0.8,1).detach()
        b = (b + epsilon * b.grad.data).clamp(0.05,0.1).detach()

    return (a * x + b * tum_sound).clamp(-1, 1)

In [ ]:
valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)

accuracies = []
examples = []

# Run test for each epsilon
epsilons = [.3]

for eps in epsilons:
    acc, ex = testAttack(model, valid_loader, insertion_attack, eps, num_iter=10)
    accuracies.append(acc)
    examples.append(ex)

## Interpolation attack

In [ ]:
'''
    Interpolation attack: 
        - take gradient wrt. the interpolation parameters a,b 
        
    Parameters:
        - tum_sound: sound to be inserted
        - epsilon: gradient step size
        - num_iterations: PGD iterations
        - clamping parameters for a, b (4 in total): define the max/min of interpolation volume
'''
def insertion_attack(model, x, y, epsilon, num_iter=1):
    global tum_sound
    
    x = x.clone().detach()
    a = torch.tensor(1.0).cuda() # original sound volume
    b = torch.tensor(1.0).cuda() # inserted sound volume
    tum_sound = tum_sound.cuda()
    
    for i in range(num_iter):
        a.requires_grad_()
        b.requires_grad_()
        
        loss = F.nll_loss(model(a * x + b * tum_sound), y)
        model.zero_grad()
        loss.backward()
        
        a = (a + epsilon * a.grad.data).clamp(0.8,1).detach()
        b = (b + epsilon * b.grad.data).clamp(0.05,0.1).detach()

    return (a * x + b * tum_sound).clamp(-1, 1)

In [ ]:
valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)

accuracies = []
examples = []

# Run test for each epsilon
epsilons = [.3]

for eps in epsilons:
    acc, ex = testAttack(model, valid_loader, insertion_attack, eps, num_iter=10)
    accuracies.append(acc)
    examples.append(ex)

    Epsilon: 0.3	Iterations: 10	Test Accuracy = 1489 / 1687 = 0.8826318909306461

In [ ]:
eps = -1
adversarial_samples = {}

for sample in examples[eps]:
    if sample[1] == 1: # ie. previously correctly classified as EM
        print("found good adversarial sample: ")
        example_id = sample[0]
        print("\tSample id: " + str(example_id))
        adversarial_samples.update({example_id : sample})

In [ ]:
sample = adversarial_samples[1149] #random.sample(list(adversarial_samples.values()),1)[0]
print(sample[0])
original = validation_dataset[sample[0]][0]
adversarial = sample[-1]

ipd.display(ipd.Audio(original,    rate=FIXED_SAMPLE_RATE, normalize=False))
ipd.display(ipd.Audio(adversarial, rate=FIXED_SAMPLE_RATE, normalize=False))

## Volume attacks

In [ ]:
def volume_attack(model, x, y, epsilon, num_iter=1):
    x = x.clone().detach()
    a = torch.tensor(1.0).cuda()
    
    for i in range(num_iter):
        a.requires_grad_()
        
        loss = F.nll_loss(model(a * x), y)
        model.zero_grad()
        loss.backward()
        
        a = (a + epsilon * a.grad.data).clamp(0.2,2).detach()
    
    return (a * x).clamp(-1, 1)

In [ ]:
valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)

accuracies = []
examples = []

# Run test for each epsilon
epsilons = [.3]

for eps in epsilons:
    acc, ex, _, _ = testAttack(model, valid_loader, volume_attack, eps, num_iter=10)
    accuracies.append(acc)
    examples.append(ex)

    Epsilon: 0.3	Iterations: 10	Test Accuracy = 1387 / 1687 = 0.8221695317131001

In [ ]:
eps = -1
adversarial_samples = {}

for sample in examples[eps]:
    if sample[1] == 1: # ie. previously correctly classified as EM
        print("found good adversarial sample: ")
        example_id = sample[0]
        print("\tSample id: " + str(example_id))
        adversarial_samples.update({example_id : sample})

In [ ]:
sample = random.sample(list(adversarial_samples.values()),1)[0]
print(sample[0])
original = validation_dataset[sample[0]][0]
adversarial = sample[-1]

ipd.display(ipd.Audio(original,    rate=FIXED_SAMPLE_RATE, normalize=False))
ipd.display(ipd.Audio(adversarial, rate=FIXED_SAMPLE_RATE, normalize=False))

## Simple PGD (SPGD) 

In [ ]:
def spgd_attack(model, x, y, epsilon, num_iter=1):
    
    perturbed_sample = x
    
    for i in range(num_iter):
        perturbed_sample = perturbed_sample.clone().detach().requires_grad_(True).cuda()
        loss = F.nll_loss(model(perturbed_sample), y)
        model.zero_grad()
        loss.backward()
        
        perturbed_sample = perturbed_sample + epsilon * perturbed_sample.grad.data
        perturbed_sample = (perturbed_sample).clamp(-1, 1) # simple clamp projection

    return perturbed_sample

In [ ]:
valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)

accuracies = []
examples = []

# Run test for each epsilon
epsilons = [.05, 0.1]

for eps in epsilons:
    print(eps)
    acc, ex = testAttack(model, valid_loader, spgd_attack, eps, num_iter=20)
    accuracies.append(acc)
    examples.append(ex)

     30 min for 20 iterations each
     
     Epsilon: 0.01	Iterations: 20	Test Accuracy = 1466 / 1687 = 0.8689982216953172
     Epsilon: 0.05	Iterations: 20	Test Accuracy = 1367 / 1687 = 0.8103141671606402
     Epsilon: 0.1	Iterations: 20	Test Accuracy = 1267 / 1687 = 0.7510373443983402

In [ ]:
eps = -1
adversarial_samples = {}

for sample in examples[eps]:
    if sample[1] == 1: # ie. previously correctly classified as EM
        print("found good adversarial sample: ")
        example_id = sample[0]
        print("\tSample id: " + str(example_id))
        adversarial_samples.update({example_id : sample})

In [ ]:
sample = adversarial_samples[9] #random.sample(list(adversarial_samples.values()),1)[0]
print(sample[0])
original = validation_dataset[sample[0]][0]
adversarial = sample[-1]

ipd.display(ipd.Audio(original,    rate=FIXED_SAMPLE_RATE, normalize=False))
ipd.display(ipd.Audio(adversarial, rate=FIXED_SAMPLE_RATE, normalize=False))

## FGSM method

In [ ]:
    
    
        final_acc = correct/float(len(data_loader))
        print("Epsilon: {}\tIterations: {}\tTest Accuracy = {} / {} = {}".format(epsilon, num_iter, correct, len(data_loader), final_acc))

        return final_acc, adv_examples, success, correct

In [90]:
valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0, 'num_iter': 1}
fgsm = FGSM()
fgsm.attack(model, valid_loader, attack_parameters)

{'epsilon': 0, 'num_iter': 1}


  0%|          | 0/1687 [00:00<?, ?it/s]


NameError: name 'torch' is not defined

In [34]:
valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)

accuracies = []
examples = []

# Run test for each epsilon
epsilons = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1] 
_, _, _, TPTN = testAttack(model, valid_loader, fgsm_attack, epsilon=0, num_iter=1)

print("Start attack")
for eps in epsilons:
    acc, ex, success, _ = testAttack(model, valid_loader, fgsm_attack, eps, num_iter=1)
    print("\tSuccess Rate: {} / {} = {}".format(success, TPTN, success/float(TPTN)))
    accuracies.append(acc)
    examples.append(ex)
    print()

100%|██████████| 1687/1687 [02:36<00:00, 10.75it/s]


Epsilon: 0	Iterations: 1	Test Accuracy = 1503 / 1687 = 0.8909306461173682
Start attack


100%|██████████| 1687/1687 [02:40<00:00, 10.51it/s]


Epsilon: 0.001	Iterations: 1	Test Accuracy = 1431 / 1687 = 0.8482513337285121
	Success Rate: 72 / 1503 = 0.04790419161676647



 17%|█▋        | 281/1687 [00:26<02:10, 10.74it/s]


KeyboardInterrupt: 

    Epsilon: 0	Iterations: 1	Test Accuracy = 1503 / 1687 = 0.8909306461173682
    Epsilon: 0.001	Iterations: 1	Test Accuracy = 1431 / 1687 = 0.8482513337285121
        Success Rate: 72 / 1503 = 0.04790419161676647
    Epsilon: 0.002	Iterations: 1	Test Accuracy = 1357 / 1687 = 0.8043864848844102
        Success Rate: 146 / 1503 = 0.09713905522288756
    Epsilon: 0.005	Iterations: 1	Test Accuracy = 1104 / 1687 = 0.6544161232957914
        Success Rate: 399 / 1503 = 0.2654690618762475
    Epsilon: 0.01	Iterations: 1	Test Accuracy = 891 / 1687 = 0.5281564908120925
        Success Rate: 612 / 1503 = 0.40718562874251496
    Epsilon: 0.02	Iterations: 1	Test Accuracy = 726 / 1687 = 0.4303497332542976
        Success Rate: 777 / 1503 = 0.5169660678642715
    Epsilon: 0.05	Iterations: 1	Test Accuracy = 863 / 1687 = 0.5115589804386484
        Success Rate: 640 / 1503 = 0.4258150365934797
    Epsilon: 0.1	Iterations: 1	Test Accuracy = 982 / 1687 = 0.5820983995257855
        Success Rate: 521 / 1503 = 0.3466400532268796
    Epsilon: 0.2	Iterations: 1	Test Accuracy = 882 / 1687 = 0.5228215767634855
        Success Rate: 621 / 1503 = 0.41317365269461076
    Epsilon: 0.5	Iterations: 1	Test Accuracy = 793 / 1687 = 0.4700652045050385
        Success Rate: 710 / 1503 = 0.47238855622089154
    Epsilon: 1	Iterations: 1	Test Accuracy = 762 / 1687 = 0.45168938944872555
        Success Rate: 741 / 1503 = 0.4930139720558882

72, 146, 399, 612, 777, 640, 521, 621, 710, 741

In [ ]:
eps = -1
adversarial_samples = {}

for sample in examples[eps]:
    if sample[1] == 1: # ie. previously correctly classified as EM
        print("found good adversarial sample: ")
        example_id = sample[0]
        print("\tSample id: " + str(example_id))
        adversarial_samples.update({example_id : sample})

In [ ]:
sample = random.sample(list(adversarial_samples.values()),1)[0]
print(sample[0])
original = validation_dataset[sample[0]][0]
adversarial = sample[-1]

ipd.display(ipd.Audio(original,    rate=FIXED_SAMPLE_RATE))
ipd.display(ipd.Audio(adversarial, rate=FIXED_SAMPLE_RATE, normalize=False))

In [ ]:
drawSpec(audio2spec(torch.tensor(original)))
drawSpec(audio2spec(torch.tensor(adversarial)))

# Misc

In [ ]:
## exec break